In [38]:
import backtrader as bt
import numpy as np
import pandas as pd
import yfinance as yf
import backtrader.feeds as btfeeds
import backtrader.analyzers as btanalyzers

In [39]:
class PairTradingStrategy(bt.Strategy):
    params = (
        ('n_period', 230),
        ('threshold', 2.25),
        ('threshold_buffer', 1.125),
        ('palpha', 0.5),
        ('cutoff', 0.6),
        ('take_profit', 0.0),
    )

    def __init__(self):
        self.state = 0  # 1: short A, long B; -1: long A, short B; 0: no position
        self.data_a = self.datas[0]
        self.data_b = self.datas[1]
        self.close_a = self.data_a.close
        self.close_b = self.data_b.close
        self.account_balance = self.broker.get_cash()
        self.open_balance = self.account_balance

    def calculate_kfactor(self, close_a, close_b):
        sum_ab = np.sum(close_a * close_b)
        sum_b2 = np.sum(close_b * close_b)
        return sum_ab / sum_b2

    def calculate_zscore(self, spread):
        mean = np.mean(spread)
        std_dev = np.std(spread)
        return (spread[-1] - mean) / std_dev if std_dev > 0 else 0

    def next(self):
        # Update account balance dynamically
        self.account_balance = self.broker.get_cash()
        
        if len(self.close_a) < self.params.n_period:
            return
    
        close_a = np.array(self.close_a.get(size=self.params.n_period))
        close_b = np.array(self.close_b.get(size=self.params.n_period))
        kfactor = self.calculate_kfactor(close_a, close_b)
        spread = close_a - kfactor * close_b
        zscore = self.calculate_zscore(spread)
    
        # Display current metrics
        print(f"Z-score: {zscore}, K-factor: {kfactor}, State: {self.state}, Balance: {self.account_balance}")
    
        # Trading logic
        if self.state == 0:  # No position
            if zscore > self.params.threshold and zscore < (self.params.threshold + self.params.threshold_buffer / 2):
                lot_a = self.account_balance * self.params.palpha
                lot_b = lot_a * kfactor
                self.sell(data=self.data_a, size=lot_a)
                self.buy(data=self.data_b, size=lot_b)
                self.state = 1
            elif zscore < -self.params.threshold and zscore > -self.params.threshold - self.params.threshold_buffer / 2:
                lot_a = self.account_balance * self.params.palpha
                lot_b = lot_a * kfactor
                self.buy(data=self.data_a, size=lot_a)
                self.sell(data=self.data_b, size=lot_b)
                self.state = -1
    
        elif self.state == 1:  # Short A, Long B
            if zscore > self.params.threshold + self.params.threshold_buffer or self.broker.getvalue() < self.params.cutoff * self.open_balance:
                self.close(self.data_a)
                self.close(self.data_b)
                self.state = 0
            elif zscore < -self.params.take_profit:
                self.close(self.data_a)
                self.close(self.data_b)
                self.state = 0
    
        elif self.state == -1:  # Long A, Short B
            if zscore < -self.params.threshold - self.params.threshold_buffer or self.broker.getvalue() < self.params.cutoff * self.open_balance:
                self.close(self.data_a)
                self.close(self.data_b)
                self.state = 0
            elif zscore > self.params.take_profit:
                self.close(self.data_a)
                self.close(self.data_b)
                self.state = 0

In [40]:
# Backtesting Setup
# if __name__ == '__main__':

cerebro = bt.Cerebro()
# Fetch data from Yahoo Finance
# Fetch data from Yahoo Finance and preprocess
def fetch_yahoo_data(ticker):
    data = yf.download(ticker, start="2010-01-01", end="2023-12-31")
    
    # Remove multi-level column names
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)
    
    # Ensure the data has the correct structure for Backtrader
    data = data.rename(columns={
        'Adj Close': 'close',
        'Open': 'open',
        'High': 'high',
        'Low': 'low',
        'Volume': 'volume'
    })
    
    # Drop unnecessary columns if any
    data = data[['open', 'high', 'low', 'close', 'volume']]
    
    # Remove timezone information from the index
    data.index = data.index.tz_localize(None)
    
    return bt.feeds.PandasData(dataname=data)
    
data_a = fetch_yahoo_data('AUDUSD=X')
data_b = fetch_yahoo_data('NZDUSD=X')

cerebro.adddata(data_a)
cerebro.adddata(data_b)

# Add strategy
cerebro.addstrategy(PairTradingStrategy)

# Set broker parameters
cerebro.broker.setcash(100000)
cerebro.broker.setcommission(commission=0.001)

# Run backtest
print("Starting Portfolio Value: ", cerebro.broker.getvalue())
results = cerebro.run()
print("Ending Portfolio Value: ", cerebro.broker.getvalue())


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Starting Portfolio Value:  100000
Z-score: 0.087496900892759, K-factor: 1.2703867278145073, State: 0, Balance: 100000.0
Z-score: 0.009559032652925153, K-factor: 1.2705207168147286, State: 0, Balance: 100000.0
Z-score: -0.09523160747184183, K-factor: 1.2706328404800142, State: 0, Balance: 100000.0
Z-score: 0.3368219701262492, K-factor: 1.2708027392458845, State: 0, Balance: 100000.0
Z-score: 0.4208466683241976, K-factor: 1.2709637562026002, State: 0, Balance: 100000.0
Z-score: 0.8324757005011586, K-factor: 1.2711528788861923, State: 0, Balance: 100000.0
Z-score: 0.5053382393587299, K-factor: 1.271290149584997, State: 0, Balance: 100000.0
Z-score: 0.5406693043032772, K-factor: 1.2714422801832517, State: 0, Balance: 100000.0
Z-score: 0.7210054912390831, K-factor: 1.2716456277254733, State: 0, Balance: 100000.0
Z-score: 0.8325633017124175, K-factor: 1.2718556257500913, State: 0, Balance: 100000.0
Z-score: 0.7542787696596218, K-factor: 1.2720297044302, State: 0, Balance: 100000.0
Z-score: 0

In [ ]:
# Add strategy with optimization
cerebro.optstrategy(
    PairTradingStrategy,
    n_period=range(50, 365, 10),  # Optimize n_period (200 to 250 in steps of 10)
    threshold=np.arange(1.0, 3.0, 0.05),  # Optimize threshold (2.0 to 3.0 in steps of 0.25)
    threshold_buffer=np.arange(0.0,2.0,0.1),
    palpha=np.arange(0.1, 10, 0.1),  # Optimize palpha
    cutoff=np.arange(0.1,2,0.1),
    take_profit=np.arange(0.0,1.0,0.1)
)

# Set broker parameters
cerebro.broker.setcash(10000)
cerebro.broker.setcommission(commission=0.001)

# Run backtest
results = cerebro.run(maxcpus=4)  # Run on a single CPU
best_strategy = max(results, key=lambda strat: strat[0].broker.getvalue())

# Display the best strategy parameters and final portfolio value
print(f"Best Ending Portfolio Value: {best_strategy[0].broker.getvalue()}")
print(f"Best Parameters: {best_strategy[0].params.__dict__}")